<a href="https://colab.research.google.com/github/PARULCHUTANIPC/parul/blob/p1/FMNIST%20KERAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [0]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=941e188b59fed8eb3215d0053166a1505827aa44833b1b296a77b63c276fb58d
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=ccc7be24d2f632162078fdc9ccec9d1d11403286f762d1889cc3b38b130a1182
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [0]:
print(tf.__version__)

2.2.0-rc1


In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [0]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [0]:
train_images=train_images/255.0
test_images=test_images/255.0

In [0]:
train_images[0].shape
len(test_images)

10000

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=20, max_value=100, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [0]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.4806 - accuracy: 0.8296 - val_loss: 0.3991 - val_accuracy: 0.8572
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3517 - accuracy: 0.8713 - val_loss: 0.3678 - val_accuracy: 0.8662
Epoch 3/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.3243 - accuracy: 0.8799 - val_loss: 0.3725 - val_accuracy: 0.8687


Epoch 1/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.3913 - accuracy: 0.8602 - val_loss: 0.2898 - val_accuracy: 0.8960
Epoch 2/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.2437 - accuracy: 0.9100 - val_loss: 0.2484 - val_accuracy: 0.9125
Epoch 3/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.1840 - accuracy: 0.9320 - val_loss: 0.2513 - val_accuracy: 0.9120


Epoch 1/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.4071 - accuracy: 0.8519 - val_loss: 0.3112 - val_accuracy: 0.8860
Epoch 2/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.2710 - accuracy: 0.8985 - val_loss: 0.2575 - val_accuracy: 0.9070
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2195 - accuracy: 0.9176 - val_loss: 0.2653 - val_accuracy: 0.9023


Epoch 1/3
1688/1688 [==============================] - 18s 10ms/step - loss: 0.4723 - accuracy: 0.8313 - val_loss: 0.3693 - val_accuracy: 0.8610
Epoch 2/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3444 - accuracy: 0.8742 - val_loss: 0.3432 - val_accuracy: 0.8725
Epoch 3/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3174 - accuracy: 0.8840 - val_loss: 0.3536 - val_accuracy: 0.8750


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4307 - accuracy: 0.8450 - val_loss: 0.3444 - val_accuracy: 0.8772
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2864 - accuracy: 0.8935 - val_loss: 0.2886 - val_accuracy: 0.8977
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2375 - accuracy: 0.9125 - val_loss: 0.2655 - val_accuracy: 0.9035


INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        64032     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1486944   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,552,746
Trainable params: 1,552,746
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 8s 148us/sample - loss: 0.1383 - accuracy: 0.9486 - val_loss: 0.2772 - val_accuracy: 0.9073
Epoch 5/10
54000/54000 [==============================] - 8s 142us/sample - loss: 0.0999 - accuracy: 0.9637 - val_loss: 0.2729 - val_accuracy: 0.9107
Epoch 6/10
54000/54000 [==============================] - 8s 144us/sample - loss: 0.0719 - accuracy: 0.9734 - val_loss: 0.3503 - val_accuracy: 0.9120
Epoch 7/10
54000/54000 [==============================] - 7s 138us/sample - loss: 0.0520 - accuracy: 0.9811 - val_loss: 0.3996 - val_accuracy: 0.9090
Epoch 8/10
54000/54000 [==============================] - 8s 142us/sample - loss: 0.0396 - accuracy: 0.9860 - val_loss: 0.4146 - val_accuracy: 0.9103
Epoch 9/10
54000/54000 [==============================] - 8s 143us/sample - loss: 0.0336 - accuracy: 0.9881 - val_loss: 0.4527 - val_accuracy: 0.9142
Epoch 10/10
54000/54000 [==========================